In [ ]:
import pandas as pd
import sys
import os
sys.path.insert(0, os.path.abspath('.'))
import download
import shapely

In [ ]:
sample = pd.read_csv('data/points_for_gsv.csv')
sample["geometry"] = sample["geometry"].apply(shapely.wkt.loads)

In [ ]:
sample.head()

In [ ]:
# Extract longitude and latitude from geometry column
sample_with_coords = sample.copy()
sample_with_coords['longitude'] = sample_with_coords['geometry'].apply(lambda geom: round(geom.x, 6))
sample_with_coords['latitude'] = sample_with_coords['geometry'].apply(lambda geom: round(geom.y, 6))

# Export to CSV
sample_with_coords.to_csv('data/sample_with_coords.csv', index=False)

In [ ]:
downloader = download.AMSDownloader()
images_path = "data/ams_images"

downloader.download_svi(images_path, input_csv_file="data/sample_with_coords.csv", start_date="2023-01-01", end_date="2023-12-31", buffer=100, distance=100)

In [ ]:
import os
import cv2
import numpy as np
from cubemap import equirec_to_cubemap
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

out_size = 640  # Size of each cubemap face

directory = "data/ams_images"
files = [f for f in os.listdir(directory) if f.endswith('.jpg')]

# load ams_pids.csv
pids_df = pd.read_csv('data/ams_images/ams_pids.csv')
pids_df.set_index('image_id', inplace=True)

def process_image(file):
    """Process a single image file"""
    try:
        equirec_image = cv2.imread(os.path.join(directory, file))
        if equirec_image is None:
            print(f"Failed to load image: {file}")
            return False
            
        car_heading = pids_df.loc[int(file.replace(".jpg",""))].heading
        faces = equirec_to_cubemap(equirec_image, out_size, car_heading)
        new_file_name = file.replace(".jpg", "")
        
        # Ensure output directory exists
        os.makedirs("data/ams_images_cubemap", exist_ok=True)
        
        cv2.imwrite(f"data/ams_images_cubemap/{new_file_name}_front.jpg", faces[3])
        cv2.imwrite(f"data/ams_images_cubemap/{new_file_name}_back.jpg", faces[1])
        
        # print(f"Processed: {file}")
        return True
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")
        return False

# Process images in parallel
max_workers = min(32, (os.cpu_count() or 1) + 4)  # Reasonable thread count
successful = 0
failed = 0

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_file = {executor.submit(process_image, file): file for file in files}
    
    # Process completed tasks
    for future in as_completed(future_to_file):
        file = future_to_file[future]
        try:
            result = future.result()
            if result:
                successful += 1
            else:
                failed += 1
        except Exception as exc:
            print(f"File {file} generated an exception: {exc}")
            failed += 1

print(f"Processing complete. Successful: {successful}, Failed: {failed}")